In [1]:
#openCV의 경우 파일 이름/경로에 한글이 들어가면 안 된다.

import cv2
import numpy as np

img_src = cv2.imread(r"img\lycoreco2.jpg", cv2.IMREAD_COLOR)
height, width = img_src.shape[:2]
cv2.imshow('src', img_src)
cv2.waitKey() #키보드를 누를때까지 종료하지마 (위의 코드까지면 금방 닫힘)
cv2.destroyAllWindows() #위 명령어 수행 후 창을 종료. 이 예제는 창이 하나지만 이 코드는 모든 창을 닫음

In [3]:
#이미지 창 두개 띄우기

img_src = cv2.imread(r"img\bocchi4.jpg", cv2.IMREAD_COLOR)
#칼라2그레이
img_src_dst = cv2.cvtColor(img_src, cv2.COLOR_BGR2GRAY)
height, width = img_src.shape[:2]
cv2.imshow('bocchi_c', img_src)
cv2.imshow('bocchi_dst', img_src_dst) #여러창 띄울거면 첫번째 인수인 이름도 달리 줘야! 아니면 하나만 떠
cv2.waitKey() 
cv2.destroyAllWindows() 

In [5]:
#matrix

import numpy as np


#병진

M1 = np.array([[1, 0, 100], [0, 1, 200]], dtype=float)


# 회전 행렬: 여기서 중심좌표를 지정한다는 건 회전변환에 중심의 평행이동까지 고려되었다는 의미

angle = 45
center = (width/2,height/2)
M2 = cv2.getRotationMatrix2D(center, angle, 1)

#y축대칭
M3 = np.array([
    [-1, 0, width],
    [0, 1, 0]
], dtype=float)

#x축 전단 변환
shx = 0.5
M4 = np.array([[1, shx, 0], [0, 1, 0]], dtype=float) # shx는 전단 계수

#스케일링
sy = 0.7
sx = 0.5
M5 = np.array([[sx, 0, 0], [0, sy, 0]], dtype=float) # sx, sy는 각 축의 크기 조정 계수

NameError: name 'width' is not defined

In [18]:
import cv2
import numpy as np


# warpAffine 함수를 사용하여 변환 행렬을 적용합니다.
dst_1 = cv2.warpAffine(img_src, M1, (width, height))
dst_2 = cv2.warpAffine(img_src, M2, (width, height))
dst_3 = cv2.warpAffine(img_src, M3, (width, height))
dst_4 = cv2.warpAffine(img_src, M4, (width, height))
dst_5 = cv2.warpAffine(img_src, M5, (width, height))

# 결과 출력
result1 = cv2.hconcat([img_src, dst_1])
result2 = cv2.hconcat([img_src, dst_2])
result3 = cv2.hconcat([img_src, dst_3])
result4 = cv2.hconcat([img_src, dst_4])
result5 = cv2.hconcat([img_src, dst_5])
cv2.imshow("result1", result1)
cv2.imshow("result2", result2)
cv2.imshow("result3", result3)
cv2.imshow("result4", result4)
cv2.imshow("result5", result5)

# cv2.imshow("result1", dst_1)
# cv2.imshow("result2", dst_2)
# cv2.imshow("result3", dst_3)
# cv2.imshow("result4", dst_4)
# cv2.imshow("result5", dst_5)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#행렬변환이 아닌 x,y 하나하나 for문으로 돌리는 경우

import cv2
import numpy as np



# 45도를 라디안으로 변환하여 코싸인값과 싸인값을 구합니다.
angle = 45
radian = angle*np.pi/180
c = np.cos(radian)
s = np.sin(radian)

# 회전변환행렬을 구성합니다.
# OpenCV의 원점이 왼쪽아래가 아니라 왼쪽위라서 [[c, -s, 0], [s, c, 0]]가 아니라
# [[c, s, 0], [-s, c, 0]]입니다.
rotation_matrix = np.array([[c, s, 0], [-s, c, 0], [0, 0, 1]], dtype=float)


dst = np.zeros(img_src.shape, dtype=np.uint8)

for y in range(height-1):
	for x in range(width-1):

		# backward mapping: 물리학의 상대속도를 생각해보자. 그리고 이미지는 좌표가 아니라 좌표상의 함수, 즉 scala field(section)다.
		# 결과 이미지의 픽셀 new_p로 이동하는 입력 이미지의 
		# 픽셀 old_p의 위치를 계산합니다.
		new_p = np.array([x, y, 1])
		inv_rotation_matrix = np.linalg.inv(rotation_matrix)
		old_p = np.dot(inv_rotation_matrix, new_p)

        # new_p 위치에 계산하여 얻은 old_p 픽셀의 값을 대입합니다.
		x_,y_ = old_p[:2]
		x_ = int(x_)
		y_ = int(y_)

		# 입력 이미지의 픽셀을 가져올 수 있는 경우에만
		# 결과 이미지의 현재 위치의 픽셀로 사용합니다.
		if x_ > 0 and x_ < width and y_ > 0 and y_ < height:
			dst[y, x] = img_src[y_, x_]

			
result = cv2.hconcat([img_src, dst]) 
cv2.imshow("result", result)
cv2.waitKey(0)


-1

T1은 원점으로의 회전 변환(원좌표 - 센터), T2는 그 역변환(원좌표 + 센터)

x->x+2일때 함수 f(x)는 f(x-2)임을 생각. MxN 행렬집합은 두 벡터공간 간의 선형사상과 동형

R * T₁ = [ cosθ  sinθ  0 ] * [ 1  0  -center_x ] = [ cosθ  sinθ  -center_x*cosθ - center_y*sinθ ]
         [ -sinθ cosθ  0 ]   [ 0  1  -center_y ]   [ -sinθ cosθ  center_x*sinθ - center_y*cosθ ]
         [  0     0    1 ]   [ 0  0     1     ]   [  0     0                  1                 ]

T₂ * (R * T₁) = [ 1  0  center_x ] * [ cosθ  sinθ  -center_x*cosθ - center_y*sinθ ] 
               [ 0  1  center_y ]   [ -sinθ cosθ  center_x*sinθ - center_y*cosθ ] = [ cosθ  sinθ  (1-cosθ)center_x - sinθcenter_y ]
               [ 0  0     1     ]   [  0     0                  1                 ]   [ -sinθ cosθ  sinθcenter_x + (1-cosθ)center_y ]
                                                                                                 [  0     0                      1                 ]

M = [ cosθ  sinθ  (1-cosθ)center_x - sinθcenter_y ]
    [ -sinθ cosθ  sinθcenter_x + (1-cosθ)center_y ]
    [  0     0                      1                 ]                                                                                            


In [14]:
### 이미지 확대축소

import cv2
import numpy as np


# (x,y)에 있는 픽셀을 위해 보간법을 적용하는 함수입니다.
def BilinearInterpolation(Q11, Q12, Q21, Q22, x1, x2, y1, y2, x, y):

	P = 1/((x2-x1)*(y2-y1))*(Q11*(x2-x)*(y2-y)+Q21*(x-x1)*(y2-y)+Q12*(x2-x)*(y-y1)*Q22*(x-x1)*(y-y1))

	return P


# 0.5배 축소하는 변환행렬을 생성합니다. 
scale_factor = 0.5
new_width = int(width*scale_factor)
new_height = int(height*scale_factor)
scaling_matrix = np.array([[scale_factor, 0, 0], [0, scale_factor, 0], [0, 0, 1]])


# backward mapping을 사용하여 축소변환행렬 scaling_matrix을 이미지에 적용합니다. 
dst = np.zeros((new_height, new_width, img_src.shape[2]) , dtype=np.uint8)

for y in range(new_height):
	for x in range(new_width):

		new_p = np.array([x, y, 1])
		inv_scaling_matrix = np.linalg.inv(scaling_matrix)
		old_p = np.dot(inv_scaling_matrix, new_p)

		x_,y_ = old_p[:2]
		x_ = int(x_)
		y_ = int(y_)

		dst.itemset((y, x, 0), img_src.item(y_, x_, 0))
		dst.itemset((y, x, 1), img_src.item(y_, x_, 1))
		dst.itemset((y, x, 2), img_src.item(y_, x_, 2))


# 보간법을 적용합니다. 
dst2 = np.zeros((new_height, new_width, img_src.shape[2]) , dtype=np.uint8)

for y in range(new_height-1):
	for x in range(new_width-1):

		q11 = dst[y - 1, x - 1]
		q12 = dst[y + 1, x - 1]
		q21 = dst[y + 1, x + 1]
		q22 = dst[y - 1, x + 1]


		if dst[y, x].all() == 0:
			p = BilinearInterpolation(q11, q12, q21, q22, x - 1, x + 1, y - 1, y + 1, x, y);
	
		else:
			p = dst[y, x]

		dst2.itemset(y, x, 0, p[0])
		dst2.itemset(y, x, 1, p[1])
		dst2.itemset(y, x, 2, p[2])

			 
cv2.imshow("result", dst2)
cv2.waitKey(0)


-1

In [11]:
# 이미지 확대 축소

import cv2
import numpy as np

img_dst = np.zeros_like(img_src)

img_tmp = cv2.resize(img_src, dsize=(0,0),fx=0.7, fy=0.7, interpolation=cv2.INTER_LINEAR) #이미지 사이즈 조절 함수
h,w = img_tmp.shape[:2] #(resize된)img_tmp의 가로 세로 비율 추출
img_dst[:h,:w,:] = img_tmp #마지막에 빈 인자(:)는 칼라여부. 지금은 칼라 다 넣을거라 빈인자로
img_dst = cv2.hconcat([img_src,img_dst]) # 원래 이미지(img_src)와 변형된 이미지(검정배경 위의 이미지)를 가로로 배열해
cv2.imshow('lyco', img_dst)

cv2.waitKey()
cv2.destroyAllWindows()

변환의 종합

In [ ]:
import cv2
import numpy as np

img_src = cv2.imread(r"img\lycoreco2.jpg", cv2.IMREAD_COLOR)

#스케일링 (s)
height, width = img_src.shape[:2]
img_zero = np.zeros_like(img_src) #검정이미지
img_s1 = cv2.resize(img_src, dsize=(0,0),fx=0.7, fy=0.7, interpolation=cv2.INTER_LINEAR)
h7,w7 = img_s1.shape[:2] #변환 후의 이미지의 가로 세로 정보 get
img_zero[:h7,:w7,:] = img_s1 #zero(검정이미지)위에 s1이미지

#병진 (t가 가해져 ts)
w_move = (width - w7)//2 #원래 사이즈에서 스케일링된 사이즈 빼기 (검정 모투리에 붙은 이미지의 중심을 검정 전체의 중심으로 병진이동. 모서리가 (0,0)임에 유의. 즉 이동은 마이너스. //는 몫만 취함)
h_move = (height - h7)//2 #델타값임에 유의
M2 = np.array([[1, 0, w_move], [0, 1, h_move]], dtype=float)
img_ts = cv2.warpAffine(img_zero, M2, (width, height))

#회전 (r이 가해져 rts)
angle = 45
center=(height//2,width//2)
rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
img_rts = cv2.warpAffine(img_ts, rotation_matrix, (width, height))

#원본 이미지와 변환된 가로로 이미지 붙이기(세로는 vconcat)
img_dst = cv2.hconcat([img_src, img_rts])

#이미지 표시
cv2.imshow('lycoris', img_dst)
cv2.waitKey()
cv2.destroyAllWindows()


In [2]:
#만약 보간, backward processing 등을 무시하고 오로지 행렬곱으로만 한다면

import cv2
import numpy as np

img_src = cv2.imread(r"img\lycoreco2.jpg", cv2.IMREAD_COLOR)
height, width = img_src.shape[:2]

# 스케일링
scale_x = 0.7
scale_y = 0.7
scale_matrix = np.array([[scale_x, 0, 0],[0, scale_y, 0],[0, 0, 1]], dtype=float)

# 병진(스케일링 된 중심점으로 이동)
w_move = (width - width * scale_x) // 2
h_move = (height - height * scale_y) // 2
translation_matrix = np.array([[1, 0, w_move],[0, 1, h_move],[0, 0, 1]], dtype=float)

# 회전(회전 중심은 resize-병진이동된 이미지가 아니라 바탕의 검정 이미지)
angle = 45
center = (width // 2, height // 2)
rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
rotation_matrix = np.vstack([rotation_matrix, [0, 0, 1]]) #위 동차좌표에 대해 차원 맞추기. 근데 이 코드 주석처리해도 작동함.

# 행렬합성 (스케일s -> 병진t -> 회전r; 이때 각 행렬의 원소가 되는 파라미터를 결정하는데 유의) 
combined_matrix = rotation_matrix @ translation_matrix @ scale_matrix

# 결합된 변환 행렬 적용(마지막에 보간 인자)
img_rts = cv2.warpAffine(img_src, combined_matrix[:2, :], (width, height), flags=cv2.INTER_LINEAR) 

img_dst = cv2.hconcat([img_src, img_rts])
cv2.imshow('lycoris2', img_dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np


img = img_src
height,width = img.shape[:2]


scale_factor = 0.5
scaling_matrix = np.array([[scale_factor, 0, 0], [0, scale_factor, 0], [0, 0, 1]])
translation_matrix = np.array([[1, 0, width/4], [0, 1, height/4], [0, 0, 1]])
angle = 45
radian = angle*np.pi/180
c = np.cos(radian)
s = np.sin(radian)
center_x = width / 2
center_y = height / 2
rotation_matrix = np.array([[c, s, (1-c)*center_x-s*center_y], [-s, c, s*center_x+(1-c)*center_y], [0, 0, 1]])

# 정해진 순서대로 변환 행렬을 곱하여 하나의 행렬을 생성합니다. 
T = np.eye(3)
T = np.dot(scaling_matrix, T)
T = np.dot(translation_matrix, T)
T = np.dot(rotation_matrix, T)

dst = np.zeros((height, width, img.shape[2]) , dtype=np.uint8)

for y in range(height):
	for x in range(width):

             # 미리 구해놓은 변환행렬을 행렬곱 한번으로 적용합니다. 
             # 여기에서도 backward mapping을 사용합니다. 
		new_p = np.array([x, y, 1])
		inv_scaling_matrix = np.linalg.inv(T)
		old_p = np.dot(inv_scaling_matrix, new_p)

		x_,y_ = old_p[:2]
		x_ = int(x_)
		y_ = int(y_)

		if x_ > 0 and x_ < width and y_ > 0 and y_ < height:
			dst.itemset((y, x, 0), img.item(y_, x_, 0))
			dst.itemset((y, x, 1), img.item(y_, x_, 1))
			dst.itemset((y, x, 2), img.item(y_, x_, 2))
			 
# cv2.imshow("result", dst)
img_result = cv2.hconcat([img, dst])
cv2.imshow("2-result", img_result)
cv2.waitKey()
cv2.destroyAllWindows()